In [64]:

from PIL import Image, ImageEnhance, ImageOps, ImageFile
import numpy as np
import random
import threading, os, time
import logging
import cv2
# logger = logging.getLogger(__name__)
# ImageFile.LOAD_TRUNCATED_IMAGES = True


class DataAugmentation:


    def __init__(self):
        pass

    @staticmethod
    def openImage(image):
        return Image.open(image, mode="r")

    @staticmethod
    def randomRotation(image, label, mode=Image.BICUBIC):

        random_angle = np.random.randint(1, 360)
        return image.rotate(random_angle, mode), label.rotate(random_angle, Image.NEAREST)


    @staticmethod
    def randomCrop(image, label):

        image_width = image.size[0]
        image_height = image.size[1]
        crop_win_size = np.random.randint(40, 68)
        random_region = (
            (image_width - crop_win_size) >> 1, (image_height - crop_win_size) >> 1, (image_width + crop_win_size) >> 1,
            (image_height + crop_win_size) >> 1)
        return image.crop(random_region), label

    @staticmethod
    def randomColor(image, label):

        random_factor = np.random.randint(0, 31) / 10.
        color_image = ImageEnhance.Color(image).enhance(random_factor)
        random_factor = np.random.randint(10, 21) / 10.
        brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)
        random_factor = np.random.randint(10, 21) / 10.
        contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)
        random_factor = np.random.randint(0, 31) / 10.
        return ImageEnhance.Sharpness(contrast_image).enhance(random_factor), label

    @staticmethod
    def randomGaussian(image, label, mean=0.2, sigma=0.3):

        def gaussianNoisy(im, mean=0.2, sigma=0.3):

            for _i in range(len(im)):
                im[_i] += random.gauss(mean, sigma)
            return im

        img = np.array(image)
        width, height = img.shape[:2]
        img_r = gaussianNoisy(img[:, :, 0].flatten(), mean, sigma)
        img_g = gaussianNoisy(img[:, :, 1].flatten(), mean, sigma)
        img_b = gaussianNoisy(img[:, :, 2].flatten(), mean, sigma)
        img[:, :, 0] = img_r.reshape([width, height])
        img[:, :, 1] = img_g.reshape([width, height])
        img[:, :, 2] = img_b.reshape([width, height])
        return Image.fromarray(np.uint8(img)), label

    @staticmethod
    def saveImage(image, path):
        # open_cv_image = np.array(image)
        # open_cv_image = open_cv_image[:, :, ::-1].copy()
        # cv2.imwrite(open_cv_image,path)
        image.save(path)


def makeDir(path):
    try:
        if not os.path.exists(path):
            if not os.path.isfile(path):
                # os.mkdir(path)
                os.makedirs(path)
            return 0
        else:
            return 1
    except Exception:
        print(str(Exception))



def imageOps(func_name, image, label, img_des_path, label_des_path, img_file_name, label_file_name, times=3):
    funcMap = {"randomRotation": DataAugmentation.randomRotation,
               "randomCrop": DataAugmentation.randomCrop,
               "randomColor": DataAugmentation.randomColor,
               "randomGaussian": DataAugmentation.randomGaussian
               }

    funcshort = {"randomRotation": 'rr',
               "randomCrop": 'rcr',
               "randomColor": 'rc',
               "randomGaussian": 'rg'
               }

    for _i in range(0, times, 1):
        new_image, new_label = funcMap[func_name](image, label)
        sname = funcshort[func_name]
        DataAugmentation.saveImage(new_image, os.path.join(img_des_path, sname + str(_i) + img_file_name))
        DataAugmentation.saveImage(new_label, os.path.join(label_des_path, sname + str(_i) + label_file_name))


opsList = {"randomRotation", "randomColor", "randomGaussian"}


def threadOPS(img_path, new_img_path, label_path, new_label_path):

    img_names = os.listdir(img_path)
    label_names = os.listdir(label_path)

    n = len(img_names)

    for i in range(n):
        img_name = img_names[i]

        label_name = label_names[i]


        tmp_img_path = os.path.join(img_path, img_name)
        tmp_label_path = os.path.join(label_path, label_name)

        print(tmp_img_path)
        image = DataAugmentation.openImage(tmp_img_path)

        label = DataAugmentation.openImage(tmp_label_path)

        threadImage = [0] * 5
        _index = 0
        for ops_name in opsList:
            imageOps(ops_name,image, label, new_img_path, new_label_path, img_name,label_name)

if __name__ == '__main__':
    threadOPS(r'C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images',
              r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\train\image",
              r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\val\label",
            r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\val\label")

In [66]:
!pip install pillow opencv-python numpy


In [67]:

# ✅ Step 2: Install Required Libraries
!pip install pillow opencv-python numpy

# ✅ Step 3: Import Necessary Libraries
import os
import random
import numpy as np
from PIL import Image, ImageEnhance
import cv2

# ✅ Step 4: Define Data Augmentation Class
class DataAugmentation:
    def __init__(self):
        pass

    @staticmethod
    def openImage(image_path):
        return Image.open(image_path, mode="r")

    @staticmethod
    def randomRotation(image, label, mode=Image.BICUBIC):
        random_angle = np.random.randint(1, 360)
        return image.rotate(random_angle, mode), label.rotate(random_angle, Image.NEAREST)

    @staticmethod
    def randomCrop(image, label):
        image_width, image_height = image.size
        crop_win_size = np.random.randint(40, 68)
        random_region = (
            (image_width - crop_win_size) >> 1,
            (image_height - crop_win_size) >> 1,
            (image_width + crop_win_size) >> 1,
            (image_height + crop_win_size) >> 1,
        )
        return image.crop(random_region), label

    @staticmethod
    def randomColor(image, label):
        random_factor = np.random.randint(0, 31) / 10.0
        color_image = ImageEnhance.Color(image).enhance(random_factor)
        random_factor = np.random.randint(10, 21) / 10.0
        brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)
        random_factor = np.random.randint(10, 21) / 10.0
        contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)
        random_factor = np.random.randint(0, 31) / 10.0
        return ImageEnhance.Sharpness(contrast_image).enhance(random_factor), label

    @staticmethod
    def randomGaussian(image, label, mean=0.2, sigma=0.3):
        def gaussianNoisy(im, mean=0.2, sigma=0.3):
            for _i in range(len(im)):
                im[_i] += random.gauss(mean, sigma)
            return im

        img = np.array(image)
        width, height = img.shape[:2]
        img_r = gaussianNoisy(img[:, :, 0].flatten(), mean, sigma)
        img_g = gaussianNoisy(img[:, :, 1].flatten(), mean, sigma)
        img_b = gaussianNoisy(img[:, :, 2].flatten(), mean, sigma)
        img[:, :, 0] = img_r.reshape([width, height])
        img[:, :, 1] = img_g.reshape([width, height])
        img[:, :, 2] = img_b.reshape([width, height])
        return Image.fromarray(np.uint8(img)), label

    @staticmethod
    def saveImage(image, path):
        image.save(path)

# ✅ Step 5: Create Directory Helper Function
def makeDir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# ✅ Step 6: Define Image Processing Function
def imageOps(func_name, image, label, img_des_path, label_des_path, img_file_name, label_file_name, times=3):
    funcMap = {
        "randomRotation": DataAugmentation.randomRotation,
        "randomCrop": DataAugmentation.randomCrop,
        "randomColor": DataAugmentation.randomColor,
        "randomGaussian": DataAugmentation.randomGaussian,
    }

    funcshort = {
        "randomRotation": 'rr',
        "randomCrop": 'rcr',
        "randomColor": 'rc',
        "randomGaussian": 'rg',
    }

    for _i in range(times):
        new_image, new_label = funcMap[func_name](image, label)
        sname = funcshort[func_name]
        DataAugmentation.saveImage(new_image, os.path.join(img_des_path, sname + str(_i) + img_file_name))
        DataAugmentation.saveImage(new_label, os.path.join(label_des_path, sname + str(_i) + label_file_name))

# ✅ Step 7: Define Multi-Threaded Image Processing Function
def threadOPS(img_path, new_img_path, label_path, new_label_path):
    makeDir(new_img_path)
    makeDir(new_label_path)

    img_names = sorted(os.listdir(img_path))
    label_names = sorted(os.listdir(label_path))

    for img_name, label_name in zip(img_names, label_names):
        tmp_img_path = os.path.join(img_path, img_name)
        tmp_label_path = os.path.join(label_path, label_name)

        print(f"Processing: {tmp_img_path}")

        image = DataAugmentation.openImage(tmp_img_path)
        label = DataAugmentation.openImage(tmp_label_path)

        opsList = {"randomRotation", "randomColor", "randomGaussian"}
        for ops_name in opsList:
            imageOps(ops_name, image, label, new_img_path, new_label_path, img_name, label_name)

# ✅ Step 8: Set Paths to Your Dataset in Google Drive
img_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"
new_img_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\train\image"
label_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\train\label"
new_label_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\data\train\label"

# ✅ Step 9: Run Image Processing
threadOPS(img_path, new_img_path, label_path, new_label_path)


In [68]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img

# ✅ Define dataset directory paths (update if needed)
IMAGE_DIR = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"
MASK_DIR = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\mask"

# ✅ Image dimensions (match model input size)
IMG_SIZE = (512, 512)

# ✅ Step 1: Check if dataset directories exist
if not os.path.exists(IMAGE_DIR):
    print(f"❌ Image directory does not exist: {IMAGE_DIR}")
if not os.path.exists(MASK_DIR):
    print(f"❌ Mask directory does not exist: {MASK_DIR}")

# ✅ Step 2: List and filter valid image/mask files
def get_files(folder, valid_exts=('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.gif')):
    return sorted([f for f in os.listdir(folder) if f.lower().endswith(valid_exts)])

image_filenames = get_files(IMAGE_DIR)
mask_filenames = get_files(MASK_DIR)

print(f"🖼️ Found {len(image_filenames)} images: {image_filenames[:5]}")
print(f"🎭 Found {len(mask_filenames)} masks: {mask_filenames[:5]}")

# ✅ Step 3: Match images and masks by filename prefix
# Image files: e.g. "21_training.tif" → key: "21_training"
image_dict = {os.path.splitext(f)[0]: f for f in image_filenames}
# Mask files: e.g. "21_training_mask.gif" → remove the "_mask" suffix → key: "21_training"
mask_dict = {os.path.splitext(f)[0].replace("_mask", ""): f for f in mask_filenames}

matched_images, matched_masks = [], []
unmatched_images = []

for img_prefix, img_name in image_dict.items():
    mask_name = mask_dict.get(img_prefix, None)  # Look for a corresponding mask
    if mask_name:
        matched_images.append(img_name)
        matched_masks.append(mask_name)
    else:
        unmatched_images.append(img_name)

# ✅ Debugging: Report unmatched images
if len(matched_images) == 0:
    print("❌ No matching image-mask pairs found! Check filenames.")
    exit()

for img_name in unmatched_images:
    print(f"⚠️ No matching mask found for: {img_name}")

print(f"✅ Matched {len(matched_images)} image-mask pairs.")

# ✅ Step 4: Load and preprocess dataset
def load_data(image_dir, mask_dir, image_files, mask_files):
    images, masks = [], []

    for img_name, mask_name in zip(image_files, mask_files):
        # Construct full file paths using separate variable names
        img_file_path = os.path.join(image_dir, img_name)
        mask_file_path = os.path.join(mask_dir, mask_name)

        # Check that both files exist
        if not os.path.exists(img_file_path) or not os.path.exists(mask_file_path):
            print(f"❌ Missing file: {img_file_path} or {mask_file_path}")
            continue  # Skip if either file is missing

        # Load and normalize image
        img = load_img(img_file_path, target_size=IMG_SIZE)
        img = img_to_array(img) / 255.0

        # Load mask (grayscale), normalize, and threshold to binary
        mask = load_img(mask_file_path, color_mode="grayscale", target_size=IMG_SIZE)
        mask = img_to_array(mask) / 255.0
        mask = np.where(mask > 0.5, 1, 0)

        images.append(img)
        masks.append(mask)

    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)

# ✅ Step 5: Load dataset using matched image and mask filenames
X, Y = load_data(IMAGE_DIR, MASK_DIR, matched_images, matched_masks)

print(f"✅ Loaded {len(X)} images and {len(Y)} masks.")
print(f"📏 Image shape: {X.shape}, Mask shape: {Y.shape}")


🖼️ Found 0 images: []
🎭 Found 2243 masks: ['21_training_mask.gif', '22_training_mask.gif', '23_training_mask.gif', '24_training_mask.gif', '25_training_mask.gif']
❌ No matching image-mask pairs found! Check filenames.
✅ Matched 0 image-mask pairs.
✅ Loaded 0 images and 0 masks.
📏 Image shape: (0,), Mask shape: (0,)


In [69]:
!pip install tensorflow numpy pandas matplotlib scikit-learn opencv-python


In [1]:
# Define your paths
dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"  # Update with your dataset folder
model_save_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\model"  # Update for saving model
# weights_path = "/content/drive/MyDrive/pretrained_weights.h5"  # Update if using pretrained weights
output_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\results2.xlsx"  # Folder to save outputs


In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2


In [5]:
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [7]:
import os

print("Dataset path:", dataset_path)
print("Contents:", os.listdir(dataset_path))


Dataset path: C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images
Contents: []


In [9]:
import os
import shutil

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"
new_dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized"

class_1_path = os.path.join(new_dataset_path, "class_1")
class_2_path = os.path.join(new_dataset_path, "class_2")

# Create class folders
os.makedirs(class_1_path, exist_ok=True)
os.makedirs(class_2_path, exist_ok=True)

# Move images into respective class folders (Modify logic based on actual labels)
for file in os.listdir(dataset_path):
    if "class1" in file:  # Adjust this condition based on your dataset
        shutil.move(os.path.join(dataset_path, file), class_1_path)
    else:
        shutil.move(os.path.join(dataset_path, file), class_2_path)

print("Dataset restructured successfully!")


Dataset restructured successfully!


In [11]:
dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized"

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 690 images belonging to 2 classes.


In [13]:
import os

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"
file_list = os.listdir(dataset_path)

print(f"Total images found: {len(file_list)}")
print(file_list[:10])  # Print first 10 filenames for verification


Total images found: 0
[]


In [15]:
import os

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\images"
if os.path.exists(dataset_path):
    print("✅ Folder exists!")
    print("Contents:", os.listdir(dataset_path))
else:
    print("❌ Folder not found. Check path!")


✅ Folder exists!
Contents: []


In [17]:
import os

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized"
print("✅ Folder exists!")
print("Contents:", os.listdir(dataset_path))


✅ Folder exists!
Contents: ['class_1', 'class_2']


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized"

train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,  # ✅ Now points to the parent folder
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,  # ✅ Now points to the parent folder
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 690 images belonging to 2 classes.
Found 172 images belonging to 2 classes.


In [21]:
import os

dataset_path = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized"

total_images = sum([len(files) for _, _, files in os.walk(dataset_path)])
print("Total images found:", total_images)


Total images found: 862


In [23]:
for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        print(f"Class '{class_folder}' has {len(os.listdir(class_path))} images.")


Class 'class_1' has 65 images.
Class 'class_2' has 797 images.


In [25]:
print(os.listdir(dataset_path + "/class_1"))  # Check inside class folders
print(os.listdir(dataset_path + "/class_2"))


['1ffa92ae-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92af-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b0-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b1-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b2-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b3-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b4-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b5-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b6-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b7-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b8-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92b9-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92ba-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92bb-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92bc-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92bd-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92be-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92bf-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa92c0-8d87-11e8-9daf-6045cb817f5b..JPG', 'rr0_rc022_training.tif', 'rr0_rc023_training.tif', 'rr0_rc024_training.tif', 'rr0_rc026_training.tif', 'rr0_rc028_training.tif', 'rr0_rc029_tra

In [27]:
import imghdr

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    for file in os.listdir(class_path):
        file_path = os.path.join(class_path, file)
        if not imghdr.what(file_path):  # Check if it's an image
            print(f"❌ Non-image file found: {file_path}")


C:\Users\HP\AppData\Local\Temp\ipykernel_24196\1336093228.py:1: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [28]:
from PIL import Image
import os


for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Check if it's a valid image
            except (IOError, SyntaxError):
                print(f"❌ Corrupt or invalid image: {file_path}")


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 690 images belonging to 2 classes.
Found 172 images belonging to 2 classes.


In [33]:
print(train_generator.class_indices)  # Shows assigned class labels
print(train_generator.samples)  # Number of training images
print(val_generator.samples)  # Number of validation images


{'class_1': 0, 'class_2': 1}
690
172


In [35]:
import os
from PIL import Image

corrupted_files = []

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):  # Ensure it's a directory
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                img = Image.open(img_path)  # Try opening the image
                img.verify()  # Verify if image is not corrupted
            except Exception as e:
                corrupted_files.append(img_path)

print("Corrupted files:", corrupted_files)


Corrupted files: []


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes (class_1, class_2)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    31,490,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,509,826 (120.20 MB)

 Trainable params: 31,509,826 (120.20 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.fit(train_generator, validation_data=val_generator, epochs=10)


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.9045 - loss: 3.1783 - val_accuracy: 0.9244 - val_loss: 0.1387
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9740 - loss: 0.1107 - val_accuracy: 0.9244 - val_loss: 0.1292
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9745 - loss: 0.1324 - val_accuracy: 0.9767 - val_loss: 0.0482
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9869 - loss: 0.0380 - val_accuracy: 0.9767 - val_loss: 0.0634
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9831 - loss: 0.0562 - val_accuracy: 0.9826 - val_loss: 0.0459
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9874 - loss: 0.0457 - val_accuracy: 0.9826 - val_loss: 0.0433
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9919 - loss: 0.0411 - val_accuracy: 0.9826 - val_loss: 0.0552
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9889 - loss: 0.0281 - val_accuracy: 0.9826 - val_loss:

In [42]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 498ms/step - accuracy: 0.9895 - loss: 0.0327
Validation Accuracy: 0.9826
Validation Loss: 0.0486


In [44]:
model.save(r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\model\model.h5")
print("Model saved successfully!")


Model saved successfully!


In [68]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand batch dimension

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    return "class_1" if class_idx == 0 else "class_2"

#test_img = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized\class_1\1ffa92b8-8d87-11e8-9daf-6045cb817f5b..JPG"
test_img = r"C:\Users\HP\OneDrive\Desktop - Copy\New folder\tortuosity-master\tortuosity-master\datasets\train\organized\class_2\21_training.tif"
print("Predicted Class:", predict_image(test_img, model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Predicted Class: class_1
